### Reference    
https://www.kaggle.com/yuyougnchan/tps-nov-lightgbm-baseline/notebook    
https://www.kaggle.com/sergiosaharovskiy/tps-nov-2021-a-complete-guide    

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.calibration import calibration_curve
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_curve, RocCurveDisplay, ConfusionMatrixDisplay, confusion_matrix, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import gc
from sklearn.model_selection import train_test_split

In [ ]:
#!ls /kaggle/input/tabular-playground-series-nov-2021

In [ ]:
DATASET_PATH ="/kaggle/input/tabular-playground-series-nov-2021/"
train = pd.read_csv(DATASET_PATH + "train.csv")
test = pd.read_csv(DATASET_PATH + "test.csv")

In [ ]:
train.isnull().sum()[train.isnull().sum() != 0 ]

In [ ]:
train_df = train.drop(['id'],axis=1)
test_df = test.drop(['id'],axis=1)

In [ ]:
X = train_df.drop('target', axis=1)
Y = train_df['target']
cols = ['f'+str(i) for i in range(100)]

In [ ]:
X

In [ ]:
scaler = StandardScaler()
X[cols] = scaler.fit_transform(X[cols])
X

In [ ]:
def load_model():
    lgbm_clf = LGBMClassifier(bagging_fraction=0.2, boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.5, feature_fraction=0.15,
               importance_type='split', learning_rate=0.1, max_depth=2,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=3300,  num_leaves=21, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=5, silent=True,
               subsample=0.7, subsample_for_bin=200000,  subsample_freq=0,
                             n_jobs=-1)
    return lgbm_clf

In [ ]:
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=5, random_state=33, shuffle=True)
preds = np.zeros(test.shape[0])
auc = []
acc = []
n = 0
for train_idx, test_idx in kf.split(X[cols],Y):
    X_train, X_val = X[cols].iloc[train_idx], X[cols].iloc[test_idx]
    Y_train, Y_val = Y.iloc[train_idx], Y.iloc[test_idx]
    
    model = load_model()
    model.fit(X_train, Y_train)
    local_preds =  model.predict(test_df[cols]) # /kf.nsplits
    # print(local_preds, local_preds.shape)
    preds += local_preds/kf.n_splits
    auc.append(roc_auc_score(Y_val, model.predict_proba(X_val)[:, 1]))
    acc.append(accuracy_score(Y_val, model.predict(X_val)))

    print(f"fold: {n+1} , accuracy: {round(acc[n]*100,3)} , auc: {round(auc[n]*100,3)}")
    n+=1

In [ ]:
sub = pd.read_csv(DATASET_PATH+'/sample_submission.csv')
sub['target'] = preds
sub.to_csv('submission4.csv',index=False)
sub